In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
# input dataset
train = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")
test = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
holidays = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv', parse_dates=['date'])
oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv', parse_dates=['date'])
transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv', parse_dates=['date'])

In [3]:
# adjust date formats
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])
transactions['date'] = pd.to_datetime(transactions['date'])
holidays['date'] = pd.to_datetime(holidays['date'])
oil['date'] = pd.to_datetime(oil['date'])

# merge the dataset
train = train.merge(stores, on='store_nbr', how='left')
test = test.merge(stores, on='store_nbr', how='left')

train = train.merge(transactions, on=['date', 'store_nbr'], how='left')

train = train.merge(oil, on='date', how='left')
test = test.merge(oil, on='date', how='left')

holidays = holidays[['date', 'locale', 'transferred']].rename(columns={'locale': 'holidays_locale'})
train = train.merge(holidays, on='date', how='left', suffixes=('', '_holiday'))
test = test.merge(holidays, on='date', how='left', suffixes=('', '_holiday'))

#adjust the order of dataset
train_order = [
    'id', 'date', 'holidays_locale', 'transferred', 'dcoilwtico',
    'store_nbr', 'onpromotion', 'city', 'state', 'type', 'cluster',
    'family', 'transactions', 'sales'
]

test_order = [
    'id', 'date', 'holidays_locale', 'transferred', 'dcoilwtico',
    'store_nbr', 'onpromotion', 'city', 'state', 'type', 'cluster',
    'family'
]

train = train[train_order]
test = test[test_order]

train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Data columns (total 14 columns):
 #   Column           Dtype         
---  ------           -----         
 0   id               int64         
 1   date             datetime64[ns]
 2   holidays_locale  object        
 3   transferred      object        
 4   dcoilwtico       float64       
 5   store_nbr        int64         
 6   onpromotion      int64         
 7   city             object        
 8   state            object        
 9   type             object        
 10  cluster          int64         
 11  family           object        
 12  transactions     float64       
 13  sales            float64       
dtypes: datetime64[ns](1), float64(3), int64(4), object(6)
memory usage: 326.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----       

In [4]:
# check the missing values
print("---missing value---")
print(train.isnull().sum())

print("\n---unique value---")
for col in train:
    print(f"{col}:{train[col].nunique()}")

---missing value---
id                       0
date                     0
holidays_locale    2551824
transferred        2551824
dcoilwtico          955152
store_nbr                0
onpromotion              0
city                     0
state                    0
type                     0
cluster                  0
family                   0
transactions        249117
sales                    0
dtype: int64

---unique value---
id:3000888
date:1684
holidays_locale:3
transferred:2
dcoilwtico:994
store_nbr:54
onpromotion:362
city:22
state:16
type:5
cluster:17
family:33
transactions:4993
sales:379610
